In [ ]:
import os
from typing import List, Dict
from athina.steps import Debug, Fn, Chain, PromptExecution, Map, ExtractJsonFromString
from athina.steps.llm import PromptTemplate, PromptMessage
from athina.llms.openai_service import OpenAiService
from athina.keys import OpenAiApiKey

OpenAiApiKey.set_key(os.getenv("OPENAI_API_KEY"))
openai_service = OpenAiService()



In [ ]:
# CHAIN 1: 
# - Generate a list of car makes and models
# - Extract the car makes
# - For each car make, generate a tweet using an LLM
def generate_tweet(topic: str) -> str:
    return openai_service.chat_completion(
        messages=[
            {
                "role": "system",
                "content": f"Generate a marketing tweet about {topic}"
            }
        ],
        model="gpt-3.5-turbo")

steps = [
    # Generate a list of cars
    PromptExecution(
        llm_service=openai_service,
        template=PromptTemplate(
            messages=[
                PromptMessage(role="system", content="Generate a list of {items} structured as a JSON array.")
            ]
        ),
        model="gpt-3.5-turbo",
        model_options={},
        output_key="cars",
    ),
    ExtractJsonFromString(input_key="cars", output_key="cars"),
    Debug(),
    # Extract the make of the car for each car in the list
    Fn(
        fn=lambda cars: [car['make'] for car in cars],
        input_key="cars",
        output_key="makes"
    ),
    # Generate a marketing tweet for each car make
    Map(
        input_key="makes", 
        fn=generate_tweet,
        output_key="tweets"
        ),
    Debug(),
]

chain = Chain(sequence=steps)
result_chain = chain.run(inputs={
    "items": "4 cars with make, model, and license number."
})

makes = result_chain.get_output("makes")
tweets = result_chain.get_output("tweets")

print("context", result_chain.get_context())
print("makes: ", makes)
print("tweets: ", tweets)

tweets

In [ ]:
result_chain.get_context()

In [ ]:
# CHAIN 2: 
# - Get news from an API. (mocked function)
# - Generate summaries for the news articles.
# - Generate an email combining all the summaries.

def get_news_from_api(topic: str) -> List[Dict[str, str]]:
    return [
        {
            "title": "OpenAI has been acquired by Microsoft",
            "content": "Content 1"
        },
        {
            "title": "Google is launching a new AI research lab in Paris",
            "content": "Content 2"
        },
        {
            "title": "Meta has just released Llama 4, a multimodal AI model",
            "content": "Content 3"
        },
        {
            "title": "Apple is investing $1 billion in a new AI research center in Berlin",
            "content": "Content 4"
        },
    ]

steps = [
    # Get news articles from an API
    Fn(
        fn=get_news_from_api,
        input_key="topic",
        output_key="news"
    ),
    Debug(),
    # Extract the title for each news article
    Map(
        input_key="news",
        fn=lambda news_item: news_item['title'],
        output_key="news_titles"
    ),
    # Convert the news items array into a string
    Fn(
        input_key="news_titles",
        fn=lambda news_titles: "\n".join(news_titles),
        output_key="news_titles_str"
    ),
    Debug(),
    # Generate a summary for each news title
    PromptExecution(
        llm_service=openai_service,
        template=PromptTemplate.simple("Generate a summary for the following news titles: {news_titles_str}. Return a json array. Each element of the array should have these fields: title, summary"),
        model="gpt-3.5-turbo",
        output_key="summaries",
    ),
    # Extract the summaries as a JSON array of objects
    ExtractJsonFromString(input_key="summaries", output_key="summaries_list"),
    # Get the summary string (without the title) for each generated summary
    Map(
        input_key="summaries_list",
        fn=lambda summary: summary['summary'],
        output_key="summaries_without_titles",
    ),
    Debug(),
    # Generate an email combining all the summaries
    PromptExecution(
        llm_service=openai_service,
        template=PromptTemplate.simple("Generate a weekly roundup newsletter email from the following summaries: {summaries_without_titles}."),
        model="gpt-4o",
        output_key="email",
    ),
]

chain = Chain(sequence=steps)
result_chain = chain.run(inputs={ "topic": "AI" })
print("context", result_chain.get_context())

res = result_chain.get_output("email")

res

In [ ]:
result_chain.get_context()